In [ ]:
from datasets import load_dataset
import os
import re
import itertools
from re import finditer
import glob
import random
import fsspec
import json
from random import randint, choice
from collections import Counter
import spacy, itertools
import langid
from nltk.corpus import stopwords
import fsspec, os, gzip
from faker import Faker
from faker.providers import person, company, geo, address, ssn
from transformers import M2M100ForConditionalGeneration, M2M100Tokenizer, MarianMTModel, AutoTokenizer, pipeline
import torch
import sys
from tqdm import tqdm

In [ ]:
basic_regex = [
    ("EMAIL_ADDRESS", re.compile(
        "([a-z0-9!#$%&'*+\/=?^_`{|.}~-]+@(?:[a-z0-9](?:[a-z0-9-]*[a-z0-9])?\.)+[a-z0-9](?:[a-z0-9-]*[a-z0-9])?)")),
    ("AGE", re.compile("\S+ years old|\S+\-years\-old|\S+ year old|\S+\-year\-old")),
    # ("PHONE_NUMBER"    , re.compile('((?:(?<![\d-])(?:\+?\d{1,3}[-.\s*]?)?(?:\(?\d{3}\)?[-.\s*]?)?\d{3}[-.\s*]?\d{4}(?![\d-]))|(?:(?<![\d-])(?:(?:\(\+?\d{2}\))|(?:\+?\d{2}))\s*\d{2}\s*\d{3}\s*\d{4}(?![\d-])))')),
    # ("PHONE_NUMBER"    , re.compile('((?:(?:\+?1\s*(?:[.-]\s*)?)?(?:\(\s*(?:[2-9]1[02-9]|[2-9][02-8]1|[2-9][02-8][02-9])\s*\)|(?:[2-9]1[02-9]|[2-9][02-8]1|[2-9][02-8][02-9]))\s*(?:[.-]\s*)?)?(?:[2-9]1[02-9]|[2-9][02-9]1|[2-9][02-9]{2})\s*(?:[.-]\s*)?(?:[0-9]{4})(?:\s*(?:#|x\.?|ext\.?|extension)\s*(?:\d+)?))')),
    ("STREET_ADDRESS", re.compile(
        '\d{1,4} [\w\s]{1,20} (?:street|st|avenue|ave|road|rd|highway|hwy|square|sq|trail|trl|drive|dr|court|ct|park|parkway|pkwy|circle|cir|boulevard|blvd)\W?(?=\s|$)')),
    ("STREET_ADDRESS", re.compile('\b\d{5}(?:[-\s]\d{4})?\b')),
    ("STREET_ADDRESS", re.compile('P\.? ?O\.? Box \d+')),
    ("GOVT_ID", re.compile(
        '(?!000|666|333)0*(?:[0-6][0-9][0-9]|[0-7][0-6][0-9]|[0-7][0-7][0-2])[- ](?!00)[0-9]{2}[- ](?!0000)[0-9]{4}')),
    ("DISEASE", re.compile("diabetes|cancer|HIV|AIDS|Alzheimer's|Alzheimer|heart disease")),
    ("NORP", re.compile("upper class|middle class|working class|lower class")),
]

In [ ]:
country = {'Zimbabwe', 'Panama', 'the Soviet Union', 'Bahamas''Colombia', 'AMERICA', 'Norway',
           'The United States of America', 'Indonesia', 'Switzerland', 'U.S.A.', 'Chile', 'U.K.', "North Korea's",
           'USSR', 'Greece', 'N. Korea', 'Viet Nam', 'america', 'U.S', 'South Korea', 'Sweden', 'New Zealand',
           'The United States', 'Pakistan', 'United States', 'Holland', 'Taiwan', 'Ukraine', 'Italy', 'South Africa',
           'Turkey', 'Spain', 'the United States of America', 'Britain', 'Saudi Arabia', 'Argentina', 'Vietnam', 'Cuba',
           'Syria', 'UK', 'Belgium', 'Iran', 'Latvia', 'India', 'France', 'Great Britain', 'El Salvador', 'Australia',
           'Brazil', 'Venezuela', 'England', 'the Roman Empire', 'russia', 'Gaza', 'Costa Rica', 'Yugoslavia',
           'the United Kingdom', 'the United States', 'USA', 'North Korea', 'U.S.', 'Canada', 'US', 'America', 'China',
           'U.S.', 'Russia', 'Israel', 'Afghanistan', 'PRC', 'Mexico', 'Iraq', 'Japan', 'Germany', }
public_figures = {'Alec Baldwin', 'Stephen Harper', 'Al Gore', 'tRump', 'Trump', 'Harper', 'Justin', 'Comey', 'Obama',
                  'Gary Johnson', 'Lindsey Graham', 'Michelle Obama', 'Mark Twain', 'Jared Kushner', 'Vladimir Putin',
                  'Putin', 'Osama bin Laden', 'Fidel Castro', 'Mitch McConnell', 'Elizabeth Warren', 'Hilary Clinton',
                  'George Bush', 'John Paul II', 'Jimmy Carter', "Hillary Clinton's", 'Ann Coulter', 'Rush Limbaugh',
                  'JFK', 'Richard Nixon', 'Cameron Sellers', 'Ted Cruz', 'Richard Nixon', 'Jill Stein', 'Brad Wall',
                  'James Comey', 'Jeff Sessions', 'Loretta Lynch', 'Sean Spicer', 'Nancy Pelosi', 'Ronald Reagan',
                  'Gray Davis', 'Donald J. Trump', 'Dan Sullivan', 'George W. Bush', 'George Soros', 'Lisa Murkowski',
                  'Gore', 'Kim Jong Un', "Donald Trump's", 'Steve Bannon', 'Alex Jones', 'Paul Ryan', 'Trumps',
                  'Clintons', 'John McCain', 'Hillary', 'Obama', 'Trump', 'Clinton', 'Trudeau', 'Donald Trump', 'Bush',
                  'Jesus', 'Hillary Clinton', 'Justin Trudeau', 'Bernie Sanders', 'Reagan', 'Bill Clinton',
                  'Pope Francis', 'Hitler', 'Nixon', 'Sarah Palin', 'Barack Obama', }
NORP = {'Islam', 'Conservative', 'the Conservative Party', 'GOP', 'Christianity', 'Church', 'the Republican Party',
        'the Democratic Party', 'the Catholic Church', 'the Liberal Party', 'the Tea Party', 'Republican Party',
        'the Roman Catholic Church', 'UNION', 'MAGA', 'the Democrat Party', }
swap_org = ['Ford', 'Facebook', 'CNN', 'ABC', 'NBC', 'CBS', 'Google', 'Amazon', 'AOL', 'NCR', 'GE', 'Enron']
bantu_surnames = ["Dlamini", "Gumede", "Hadebe", "Ilunga", "Kamau", "Khoza", "Lubega", "M'Bala", "Mabaso", "Mabika",
                  "Mabizela", "Mabunda", "Mabuza", "Macharia", "Madima", "Madondo", "Mahlangu", "Maidza", "Makhanya",
                  "Malewezi", "Mamba", "Mandanda", "Mandlate", "Mangwana", "Manjate", "Maponyane", "Mapunda", "Maraire",
                  "Masango", "Maseko", "Masemola", "Masengo", "Mashabane", "Masire", "Masondo", "Masuku", "Mataka",
                  "Matovu", "Mbala", "Mbatha", "Mbugua", "Mchunu", "Mkhize", "Mofokeng", "Mokonyane", "Mutombo",
                  "Ncube", "Ndagire", "Ndhlovu", "Ndikumana", "Ndiritu", "Ndlovu", "Ndzinisa", "Ngcobo", "Nkomo",
                  "Nkosi", "Nkurunziza", "Radebe", "Tshabalala", "Tshivhumbe", "Vila"]

faker_map = dict([(a.split("_")[0], a) for a in [
    'ar_AA',
    'ar_PS',
    'ar_SA',
    'bg_BG',
    'cs_CZ',
    'de_AT',
    'de_CH',
    'de_DE',
    'dk_DK',
    'el_GR',
    'en_GB',
    'en_IE',
    'en_IN',
    'en_NZ',
    'en_TH',
    'en_US',
    'es_CA',
    'es_ES',
    'es_MX',
    'et_EE',
    'fa_IR',
    'fi_FI',
    'fr_CA',
    'fr_CH',
    'fr_FR',
    'fr_QC',
    'ga_IE',
    'he_IL',
    'hi_IN',
    'hr_HR',
    'hu_HU',
    'hy_AM',
    'id_ID',
    'it_IT',
    'ja_JP',
    'ka_GE',
    'ko_KR',
    'lt_LT',
    'lv_LV',
    'ne_NP',
    'nl_NL',
    'no_NO',
    'or_IN',
    'pl_PL',
    'pt_BR',
    'pt_PT',
    'ro_RO',
    'ru_RU',
    'sl_SI',
    'sv_SE',
    'ta_IN',
    'th_TH',
    'tr_TR',
    'tw_GH',
    'uk_UA',
    'zh_CN',
    'zh_TW']] + [('en', 'en_US')])


In [ ]:
person_swap = True
target_lang = 'id'
texts = []
ner_mappings = []
row_ids = []
domains = []
lbracket = "["
rbracket = "]"
if target_lang in ('zh', 'ja', 'ko'):
    lbracket = "[["
    rbracket = "]]"
faker_target_lang = Faker(faker_map[target_lang])
faker_target_lang.add_provider(person)
faker_target_lang.add_provider(geo)

row_id = -1
for s in tqdm(open(r"C:\Users\Administrator\pii_en.jsonl", "rb")):
    s = s.decode().strip()
    if not s: continue
    dat = json.loads(s)
    domain = dat['domain']
    ner = dat['ner']
    text = dat['text']
    if 'id' not in dat:
        row_id += 1
    else:
        row_id = int(dat['id'])
    if 'NYMEX' in text: continue
    ner = [n for n in ner if n[0] not in ("FREE", "’m", 'Social Security')]
    ner2 = []
    if ' cancer ' in text:
        ner2.append(['cancer', 'DISEASE'])
    elif ' HIV ' in text:
        ner2.append(['HIV', 'DISEASE'])
    elif ' AIDS ' in text:
        ner2.append(['AIDS', 'DISEASE'])
    elif "Alzheimer's" in text:
        ner2.append(["Alzheimer's", 'DISEASE'])
    elif "Alzheimer" in text:
        ner2.append(['Alzheimer', 'DISEASE'])
    elif 'heart disease' in text:
        ner2.append(['heart disease', 'DISEASE'])
    elif 'democractic' in text:
        ner2.append(['democractic', 'NORP'])
    elif 'democrats' in text:
        ner2.append(['democrats', 'NORP'])
    elif 'Democrats' in text:
        ner2.append(['Democrats', 'NORP'])
    elif 'democrat' in text:
        ner2.append(['democrat', 'NORP'])
    elif 'Democrat' in text:
        ner2.append(['Democrat', 'NORP'])
    elif 'republican' in text:
        ner2.append(['republican', 'NORP'])
    elif 'republicans' in text:
        ner2.append(['republicans', 'NORP'])
    elif 'Republicans' in text:
        ner2.append(['Republicans', 'NORP'])
    elif 'republican' in text:
        ner2.append(['republican', 'NORP'])
    elif 'Republican' in text:
        ner2.append(['Republican', 'NORP'])
    elif 'socialist' in text:
        ner2.append(['socialist', 'NORP'])
    elif 'Socialist' in text:
        ner2.append(['Socialist', 'NORP'])
    for item in ner:
        itemArr = item[0].split()
        if itemArr[0] in ('Association', 'Society', 'Union') or itemArr[-1] in (
                'Association', 'Society', 'Party'):
            item[1] = 'NORP'
        if item[0] == 'Enron':
            item[0] = choice(swap_org)
            text = text.replace("Enron", item[0])
            text = text.replace("@enron", '@' + item[0].lower())
            text = text.replace(" enron", ' ' + item[0].lower())
            # print (text)
        if item[0] in public_figures:
            item[1] = 'PUBLIC_FIGURE'
        elif item[0] in country:
            item[1] = 'COUNTRY'
        elif item[0] in NORP:
            item[1] = 'NORP'
        elif '@' in item[0]:
            item[1] = 'EMAIL'
        ner2.append(item)
    # col.extend ([d[0] for d in ner2 if d[1] == 'NORP'])
    context = {}
    ner_mapping = {}
    seen = {}
    text = "... " + text + " "
    text = text.replace(lbracket, "(")
    text = text.replace(rbracket, ")", )
    if person_swap:
        _idx = 0
        for item in ner2:
            if item[0] in seen: continue
            seen[item[0]] = 1
            if item[1] in ('COUNTRY', 'GPE', 'PERSON', 'GOVT_ID', 'STREET_ADDRESS',):  # ORG
                text = text.replace(" " + item[0] + " ", " *" + str(_idx) + "* ")
                text = text.replace(" " + item[0] + ",", " *" + str(_idx) + "* ,")
                text = text.replace(" " + item[0] + "'", " *" + str(_idx) + "*'")
                text = text.replace(item[0], "*" + str(_idx) + "*")
                context[item[0]] = context.get(item[0], \
                                               faker_target_lang.first_name() + " " + random.choice(
                                                   bantu_surnames) if " " in item[0] and
                                                                      item[
                                                                          1] == 'PERSON' and target_lang in (
                                                                          'yo', 'sw') else \
                                                   faker_target_lang.name() if " " in item[
                                                       0] and item[
                                                                                   1] == 'PERSON' else \
                                                       faker_target_lang.first_name() if
                                                       item[
                                                           1] == 'PERSON' else \
                                                           faker_target_lang.country() if
                                                           item[
                                                               1] == 'COUNTRY' else \
                                                               faker_target_lang.state() if
                                                               item[
                                                                   1] == 'GPE' and target_lang != 'zh' else \
                                                                   faker_target_lang.province() if
                                                                   item[
                                                                       1] == 'GPE' and target_lang == 'zh' else \
                                                                       faker_target_lang.ssn() if
                                                                       item[
                                                                           1] == 'GOVT_ID' else \
                                                                           faker_target_lang.address() if
                                                                           item[
                                                                               1] == 'STREET_ADDRESS' else \
                                                                               item[
                                                                                   0])

                ner_mapping["*" + str(_idx) + "*"] = [context[item[0]],
                                                      item[1] if item[
                                                                     1] != 'COUNTRY' else 'GPE']
                if " " in item[0]:
                    context[item[0].split()[0]] = context[item[0]].split()[0]
                    context[item[0].split()[-1]] = context[item[0]].split()[-1]
                item[0] = context[item[0]]
            else:
                text = text.replace(item[0],
                                    ' ' + str(_idx) + " " + lbracket + item[0] + rbracket)
                ner_mapping[str(_idx) + " " + lbracket] = item
            _idx += 1

    texts.append(text)
    ner_mappings.append(ner_mapping)
    row_ids.append(row_id)
    domains.append(domain)

In [ ]:
def chunks(lst, n):
    """Generate batches"""
    for i in range(0, len(lst), n):
        yield lst[i: i + n]

In [ ]:
model = M2M100ForConditionalGeneration.from_pretrained("facebook/m2m100_418M")
tokenizer = M2M100Tokenizer.from_pretrained("facebook/m2m100_418M")
model = model.to('cuda').half()
translations = []
for src_text_list in tqdm(chunks(texts, 16)):
    batch = tokenizer(src_text_list, return_tensors="pt", padding=True, truncation=True).to('cuda')
    gen = model.generate(**batch, forced_bos_token_id=tokenizer.get_lang_id(target_lang))
    outputs = tokenizer.batch_decode(gen, skip_special_tokens=True)
    translations.extend(outputs)


In [ ]:
import pickle
with open(('tr_hi'), 'wb') as f:
    pickle.dump(translations, f)

In [ ]:
rbracket = "]"
if target_lang in ('zh', 'ja', 'ko'):
    rbracket = "]]"
with open(f"pii_new_{target_lang}.jsonl", "w", encoding="utf8") as o:
    for index, trans_text in enumerate(translations):
        ner_found = []
        trans_text = trans_text.lstrip(".")
        trans_text = trans_text.strip()
        trans_text = trans_text.replace("* ", "*").replace(" *", "*").replace('"0*', '*0* ').replace(
            '"1*',
            '*1* ').replace(
            '"2*', '*2* ').replace('"3*', '*3* ').replace('"4*', '*4* '). \
            replace('*0:', '*0* ').replace('*1:', '*1* ').replace('*2:', '*2* ').replace('*3:',
                                                                                         '*3* ').replace(
            '*4:', '*4* '). \
            replace('*0 ', '*0* ').replace('*1 ', '*1* ').replace('*2 ', '*2* ').replace('*3 ',
                                                                                         '*3* ').replace(
            '*4 ', '*4* '). \
            replace(' 0*', '*0* ').replace(' 1*', '*1* ').replace('2 *', '*2* ').replace(' 3*',
                                                                                         '*3* ').replace(
            ' 4*', '*4* ')
        if trans_text.startswith('0') and not trans_text.startswith('0 ['):
            trans_text = '*0* ' + trans_text[1:]
        trans_text = trans_text.replace(". [", " [").replace(".[", " [").replace("  ", " ")
        orig_trans_text = trans_text
        for key, ner_item in ner_mappings[index].items():
            found = False
            if key in trans_text:
                found = True
            elif key.replace(" ", "") in trans_text:
                found = True
                key = key.replace(" ", "")
            elif key.lstrip('*') in trans_text:
                found = True
                key = key.lstrip('*')
            elif key.rstrip('*') in trans_text:
                found = True
                key = key.rstrip('*')
            if found:
                if key[0] == '*':
                    trans_text = trans_text.replace(key, " " + ner_item[0] + " ")
                    ner_found.append(list(ner_item))
                else:
                    trans_text2 = ""
                    for segment in trans_text.split(key):
                        if rbracket in segment:
                            entity, rest = segment.split(rbracket, 1)
                            entity = entity.strip("[]")
                            ner_found.append([entity, ner_item[1]])
                            trans_text2 += " " + entity + " " + rest
                        else:
                            trans_text2 += " " + segment
                    trans_text = trans_text2.strip()
        trans_text = trans_text.replace("*", " ").replace("[", " ").replace("]", " ").replace(
                " .",
                ".").replace(" ,",
                             ",").replace(
                "  ", " ").replace("  ", " ").strip()
        if target_lang in ('zh', 'ja', 'ko'):
            trans_text.replace(" ", "")
            trans_text.strip('#.')
        if ner_found:
            j = {'text': trans_text, 'ner': ner_found, 'domain': domains[index],
                     'id': row_ids[index],
                     'lang': target_lang}
            print(j)
            
            o.write(json.dumps(j) + "\n")